# Exploration et nettoyage famille decès mortalité 2026 : cette table est historique et nationale (1901-2025): banque de séries historiques INSEE. Elle contient plusieurs indicateurs de mortalité.

In [89]:
import pandas as pd
from pathlib import Path

raw_path = Path("../data/raw")
processed_path = Path("../data/processed")

processed_path.mkdir(exist_ok=True)

In [90]:
print("Raw exists:", raw_path.exists())
print("Processed exists:", processed_path.exists())

Raw exists: True
Processed exists: True


In [91]:
xls = pd.ExcelFile(raw_path / "famille_DECES-MORTALITE_30012026.xlsx")
xls.sheet_names

/home/celin/.pyenv/versions/sante-territoires/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


['valeurs_mensuelles', 'valeurs_annuelles', 'caractéristiques', 'codes']

In [92]:
df_fam = pd.read_excel(raw_path / "famille_DECES-MORTALITE_30012026.xlsx", sheet_name="valeurs_annuelles")
df_fam.head()

,Libellé,idBank,Dernière mise à jour,Période,1901,1902,1903,1904,1905,1906,...,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025
0,Décès domiciliés par région - Autres : décès e...,869058,08/10/2015 00:00,NaN,(O),(O),(O),(O),(O),(O),...,(O),(O),(O),(O),(O),(O),(O),(O),(O),(O)
1,Décès domiciliés par région - France métropoli...,1780755,18/04/2016 00:00,NaN,(O),(O),(O),(O),(O),(O),...,(O),(O),(O),(O),(O),(O),(O),(O),(O),(O)
2,Taux brut de mortalité - Ensemble - France mét...,1745179,04/04/2025 09:16,NaN,(O),(O),(O),(O),(O),(O),...,9,9.2,9.2,9.2,10,9.8,10,9.4,9.5,(O)
3,Taux brut de mortalité - Ensemble - France (in...,1745178,04/04/2025 09:16,NaN,(O),(O),(O),(O),(O),(O),...,8.9,9.1,9.1,9.1,9.9,9.7,9.9,9.4,9.4,(O)
4,Espérance de vie à la naissance - Hommes - Fra...,10536466,04/04/2025 11:59,NaN,(O),(O),(O),(O),(O),(O),...,79.3,79.5,79.6,79.8,79.2,79.3,79.4,80,80.1,(O)


In [93]:
df_fam.columns

Index(['Libellé', 'idBank', 'Dernière mise à jour', 'Période', '1901', '1902',
       '1903', '1904', '1905', '1906',
       ...
       '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2024',
       '2025'],
      dtype='object', length=129)

In [94]:
df_fam.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1897 entries, 0 to 1896
Columns: 129 entries, Libellé to 2025
dtypes: float64(1), int64(1), object(127)
memory usage: 1.9+ MB


Ne garder que le taux brut de mortalité

In [95]:
# Garder uniquement taux brut ensemble
df_fam = df_fam[
    (df_fam["Libellé"].str.contains("Taux brut de mortalité", case=False, na=False)) &
    (df_fam["Libellé"].str.contains("Ensemble", case=False, na=False))
].copy()

# Extraire territoire proprement
df_fam["territoire"] = df_fam["Libellé"].str.split(" - ").str[2]

# Supprimer les séries arrêtées
df_fam = df_fam[df_fam["territoire"] != "Série arrêtée"]

# Filtrer pour n'afficher que les années de 2015 à 2024 pour analyser l'évolution

In [96]:
annees = [str(y) for y in range(2015, 2024)]

df_period = df_fam[["territoire"] + annees].copy()
df_period.head()

,territoire,2015,2016,2017,2018,2019,2020,2021,2022,2023
2,France métropolitaine,9,9,9.2,9.2,9.2,10,9.8,10,9.4
3,France (inclus Mayotte à partir de 2014),8.9,8.9,9.1,9.1,9.1,9.9,9.7,9.9,9.4
1193,Ain,7.6,7.5,7.8,7.7,7.6,8.9,8,8.1,7.7
1194,Aisne,10.7,10.6,10.7,10.9,11,12.1,11.9,11.8,11.1
1195,Allier,13,13.2,13,13,13.3,14.4,13.8,14,13.7


# Transformer table large en table longue pour Big Query

In [97]:
df_long = df_period.melt(
    id_vars=["territoire"],
    var_name="annee",
    value_name="taux_brut_pour_1000"
)
df_long.head()

,territoire,annee,taux_brut_pour_1000
0,France métropolitaine,2015,9
1,France (inclus Mayotte à partir de 2014),2015,8.9
2,Ain,2015,7.6
3,Aisne,2015,10.7
4,Allier,2015,13


In [98]:
df_long["taux_brut_pour_1000"] = (
    df_long["taux_brut_pour_1000"]
    .astype(str)
    .str.replace(",", ".", regex=False)
    .str.extract(r"([0-9.]+)")
    .astype(float)
)

In [99]:
df_long.duplicated(subset=["territoire","annee"]).sum()

np.int64(0)

Filtrer sur France, Occitanie et ses départements pour analyser les évolutions territoriales

In [100]:
df_long["territoire"].unique()

array(['France métropolitaine',
       'France (inclus Mayotte à partir de 2014)', 'Ain', 'Aisne',
       'Allier', 'Alpes-de-Haute-Provence', 'Hautes-Alpes',
       'Alpes-Maritimes', 'Ardèche', 'Ardennes', 'Ariège', 'Aube', 'Aude',
       'Aveyron', 'Bouches-du-Rhône', 'Calvados', 'Cantal', 'Charente',
       'Charente-Maritime', 'Cher', 'Corrèze', "Côte-d'Or",
       "Côtes-d'Armor", 'Creuse', 'Dordogne', 'Doubs', 'Drôme', 'Eure',
       'Eure-et-Loir', 'Finistère', 'Corse-du-Sud', 'Haute-Corse', 'Gard',
       'Haute-Garonne', 'Gers', 'Gironde', 'Hérault', 'Ille-et-Vilaine',
       'Indre', 'Indre-et-Loire', 'Isère', 'Jura', 'Landes',
       'Loir-et-Cher', 'Loire', 'Haute-Loire', 'Loire-Atlantique',
       'Loiret', 'Lot', 'Lot-et-Garonne', 'Lozère', 'Maine-et-Loire',
       'Manche', 'Marne', 'Haute-Marne', 'Mayenne', 'Meurthe-et-Moselle',
       'Meuse', 'Morbihan', 'Moselle', 'Nièvre', 'Nord', 'Oise', 'Orne',
       'Pas-de-Calais', 'Puy-de-Dôme', 'Pyrénées-Atlantiques',
      

In [101]:
departements_occitanie = [
    "Ariège", "Aude", "Aveyron", "Gard", "Haute-Garonne",
    "Gers", "Hérault", "Lot", "Lozère",
    "Hautes-Pyrénées", "Pyrénées-Orientales",
    "Tarn", "Tarn-et-Garonne"
]

In [102]:
df_long = df_long[
    (df_long["territoire"] == "Occitanie") |
    (df_long["territoire"] == "France métropolitaine") |
    (df_long["territoire"].isin(departements_occitanie))
]

In [103]:
df_long["territoire"].unique()

array(['France métropolitaine', 'Ariège', 'Aude', 'Aveyron', 'Gard',
       'Haute-Garonne', 'Gers', 'Hérault', 'Lot', 'Lozère',
       'Hautes-Pyrénées', 'Pyrénées-Orientales', 'Tarn',
       'Tarn-et-Garonne', 'Occitanie'], dtype=object)

In [104]:
df_long.head()

,territoire,annee,taux_brut_pour_1000
0,France métropolitaine,2015,9.0
10,Ariège,2015,11.7
12,Aude,2015,11.4
13,Aveyron,2015,12.6
32,Gard,2015,9.9


In [105]:
df_long.info()

<class 'pandas.core.frame.DataFrame'>
Index: 135 entries, 0 to 1048
Data columns (total 3 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   territoire           135 non-null    object 
 1   annee                135 non-null    object 
 2   taux_brut_pour_1000  135 non-null    float64
dtypes: float64(1), object(2)
memory usage: 4.2+ KB


In [107]:
df_long["annee"] = df_long["annee"].astype("Int64")

In [108]:
df_long.info()

<class 'pandas.core.frame.DataFrame'>
Index: 135 entries, 0 to 1048
Data columns (total 3 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   territoire           135 non-null    object 
 1   annee                135 non-null    Int64  
 2   taux_brut_pour_1000  135 non-null    float64
dtypes: Int64(1), float64(1), object(1)
memory usage: 4.4+ KB


In [109]:
df_long.describe()

,annee,taux_brut_pour_1000
count,135.0,135.000000
mean,2019.0,11.431111
std,2.591605,1.842592
min,2015.0,6.800000
25%,2017.0,10.050000
50%,2019.0,11.900000
75%,2021.0,12.900000
max,2023.0,14.400000


In [110]:
df_long.groupby("territoire")["taux_brut_pour_1000"].mean().sort_values()

territoire
Haute-Garonne             7.122222
France métropolitaine     9.422222
Hérault                   9.422222
Occitanie                10.266667
Gard                     10.311111
Tarn-et-Garonne          10.600000
Tarn                     11.588889
Aude                     12.233333
Pyrénées-Orientales      12.300000
Ariège                   12.633333
Aveyron                  12.844444
Hautes-Pyrénées          12.966667
Gers                     13.044444
Lozère                   13.122222
Lot                      13.588889
Name: taux_brut_pour_1000, dtype: float64

In [112]:
df_long = df_long.sort_values(["territoire","annee"])
df_long.head()

,territoire,annee,taux_brut_pour_1000
10,Ariège,2015,11.7
127,Ariège,2016,12.0
244,Ariège,2017,12.7
361,Ariège,2018,12.1
478,Ariège,2019,12.7


In [113]:
df_long.to_csv(processed_path / "mortalite_nationale_historique.csv", index=False)